This went quite well - got 0.6995 on private score, 0.70083 on the public one. It probably could be tweaked as the neural network at the end does the difference. The other competitors mentioned scaling the target variable to [0-1] range and were saying it allowed them to go down a bit with their MSE, but given that this task got me tired as no other, for now I'm leaving the solution as is.

In [1]:
import os
import csv
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras

In [2]:
train_full = pd.read_csv('data/train.csv')
train_ids = train_full[['id']].to_numpy(dtype=int)
xs = train_full.drop('id', axis=1).to_numpy()
ys = train_full[['target']].to_numpy()
test_full = pd.read_csv('data/test.csv')
ids_test = test_full[['id']].to_numpy(dtype=int)
test = test_full.drop('id', axis=1).to_numpy()
split = int(xs.shape[0] * .8)
X_train = xs[:split, :-1]
y_train = ys[:split, -1]
ids_train = train_ids[:split, 0]
X_valid = xs[split:, :-1]
y_valid = ys[split:, -1]
ids_valid = train_ids[split:, 0]

In [3]:
class SwapNoise(keras.layers.Layer):
    def __init__(self, ratio=0.15, col_to_apply=None, **kwargs):
        super().__init__(**kwargs)
        self.ratio = ratio
        self.col_to_apply = col_to_apply

    def call(self, inputs, training=None):
        if training:
            noisy_inputs = tf.map_fn(lambda x: SwapNoise._add_swap_noise(x, ratio=self.ratio, col_to_apply=self.col_to_apply), inputs)
            return noisy_inputs
        else:
            return inputs

    @staticmethod
    def _add_swap_noise(X, ratio=.15, col_to_apply=None, return_mask=False):
        if col_to_apply is None:
            col_to_apply = []

        shape = tf.shape(X)
        obfuscation_mask = tf.cast(
            tf.random.stateless_binomial(
                shape=shape,
                seed=(1, 2),
                counts=1,
                probs=tf.fill(shape, ratio)),
            dtype=tf.float32)

        if col_to_apply:
            column_mask = np.zeros(X.shape, dtype=np.float32)
            column_mask[col_to_apply] = 1
            obfuscation_mask *= column_mask

        shuffled_rows = tf.random.shuffle(tf.range(tf.shape(X)[0]))
        obfuscated_X = tf.where(obfuscation_mask == 1, tf.gather(X, shuffled_rows), X)

        if return_mask:
            return obfuscated_X, obfuscation_mask

        return obfuscated_X

In [4]:
regressor = keras.Sequential([
    SwapNoise(ratio=.1),
    keras.layers.Dense(256),
    keras.layers.PReLU(),
    keras.layers.Dropout(.1),
    keras.layers.Dense(256),
    keras.layers.PReLU(),
    keras.layers.Dropout(.1),
    keras.layers.Dense(256),
    keras.layers.PReLU(),
    keras.layers.Dropout(.1),
    keras.layers.Dense(256),
    keras.layers.PReLU(),
    keras.layers.Dropout(.1),
    keras.layers.Dense(1),
    keras.layers.PReLU()
])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=30,
                                               min_delta=1e-6)
regressor.compile(optimizer='adam', loss='mse')

history = regressor.fit(X_train, y_train, epochs=300, batch_size=64,
           callbacks=[
               early_stopping,
               keras.callbacks.ModelCheckpoint(
                   filepath='saved_models/dnn_regressor_after_dae{epoch}',
                   save_best_only=True)],
           validation_data=(X_valid, y_valid))

Epoch 1/300
3750/3750 [==============================] - 112s 29ms/step - loss: 0.8305 - val_loss: 0.5604
Epoch 2/300
3750/3750 [==============================] - 107s 28ms/step - loss: 0.5827 - val_loss: 0.5242
Epoch 3/300
3750/3750 [==============================] - 106s 28ms/step - loss: 0.5691 - val_loss: 0.5216
Epoch 4/300
3750/3750 [==============================] - 108s 29ms/step - loss: 0.5588 - val_loss: 0.5327
Epoch 5/300
3750/3750 [==============================] - 109s 29ms/step - loss: 0.5532 - val_loss: 0.5325
Epoch 6/300
3750/3750 [==============================] - 111s 30ms/step - loss: 0.5498 - val_loss: 0.5184
Epoch 7/300
3750/3750 [==============================] - 107s 29ms/step - loss: 0.5495 - val_loss: 0.5327
Epoch 8/300
3750/3750 [==============================] - 102s 27ms/step - loss: 0.5480 - val_loss: 0.5260
Epoch 9/300
3750/3750 [==============================] - 108s 29ms/step - loss: 0.5450 - val_loss: 0.5210
Epoch 10/300
3750/3750 [======================

In [4]:
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomTreesEmbedding
from sklearn.metrics import mean_squared_error
from scipy.sparse._csr import csr_matrix

from hyperopt import fmin, hp, tpe, STATUS_OK
from hyperopt.pyll.base import scope
from typing import Callable

In [5]:
def rte_objective(search_space: dict) -> dict:
    rte = RandomTreesEmbedding(**search_space)
    ridge = Ridge(alpha=3000)

    rte.fit(X_train)

    X_train_transformed = rte.transform(X_train)

    ridge.fit(X_train_transformed, y_train)

    X_valid_transformed = rte.transform(X_valid)
    y_pred = ridge.predict(X_valid_transformed)
    accuracy = mean_squared_error(y_valid, y_pred)

    return {'loss': accuracy, 'status': STATUS_OK}


def find_params(search_space: dict, get_objective: Callable) -> dict:
    algorithm = tpe.suggest
    best_params = fmin(
        fn=get_objective,
        space=search_space,
        algo=algorithm,
        max_evals=100)

    return best_params

In [12]:
rte_params = find_params({
    'n_estimators': scope.int(hp.quniform('n_estimators', 100, 2000, 50)),
    'max_depth': scope.int(hp.quniform('max_depth', 5, 10, 1)),
    'min_samples_split': scope.int(hp.quniform('min_samples_split', 50, 200, 50)),
    'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 25, 200, 25)),
    'n_jobs': -2}, rte_objective)

100%|██████████| 100/100 [3:32:32<00:00, 127.52s/trial, best loss: 0.4892925067882283] 


In [6]:
import xgboost as xgb


def xgb_objective(search_space: dict) -> dict:
    regressor = xgb.XGBRegressor(**search_space)

    regressor.fit(X_train, y_train)

    y_pred = regressor.predict(X_valid)
    accuracy = mean_squared_error(y_valid, y_pred)

    return {'loss': accuracy, 'status': STATUS_OK}

In [21]:
xgb_params = find_params({
    'max_depth': scope.int(hp.quniform('max_depth', 1, 10, 1)),
    'gamma': hp.uniform ('gamma', 0, 1),
    'colsample_bytree' : hp.uniform('colsample_bytree', 0, 1),
    'min_child_weight' : hp.uniform('min_child_weight', 0, 5),
    'learning_rate': hp.uniform('learning_rate', 0, .15),
    'random_state': 5,
    'max_bin' : scope.int(hp.quniform('max_bin', 200, 550, 1))}, xgb_objective)

100%|██████████| 100/100 [21:08<00:00, 12.69s/trial, best loss: 0.48856797903425425]


In [7]:
regressor = xgb.XGBRegressor(
    colsample_bytree=0.838418115820081,
    gamma=0.34980482863950296,
    learning_rate=0.07097890845615876,
    max_bin=367,
    max_depth=10,
    min_child_weight=1.0019995276220504)

regressor.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.838418115820081, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.34980482863950296, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.07097890845615876, max_bin=367,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=1.0019995276220504, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=None, ...)

In [8]:
rte = RandomTreesEmbedding(max_depth=5, min_samples_leaf=100, min_samples_split=200, n_estimators=1000)
ridge = Ridge(alpha=3000)

rte.fit(X_train)

X_train_transformed = rte.transform(X_train)
X_valid_transformed = rte.transform(X_valid)

ridge.fit(X_train_transformed, y_train)

Ridge(alpha=3000)

In [9]:
def create_file(name: str, xs_trans: csr_matrix, xs_old: np.ndarray, ids: np.ndarray, ys: np.ndarray = None):
    col_names = \
        ['id'] +\
        [f'cat{idx}' for idx in range(xs_trans.shape[1])] +\
        [f'cont{idx}' for idx in range(xs_old.shape[1])]

    if ys is not None:
        col_names += ['target']

    with open(os.path.join('data', f'{name}_enhanced.csv'), 'w') as file:
        writer = csv.writer(file, delimiter=',')

        writer.writerow(col_names)

        for i in range(0, xs_trans.shape[0]):
            enhanced_repr = xs_trans.getrow(i).toarray()[0]
            enhanced_repr = enhanced_repr.reshape(enhanced_repr.shape[0], 1)
            old_features = xs_old[i, :]
            old_features = old_features.reshape(old_features.shape[0], 1)
            tall_repr = np.vstack([enhanced_repr, old_features])
            wide_repr = tall_repr.reshape(1, tall_repr.shape[0]).tolist()[0]
            user_id = [ids[i, 0]] if ids.ndim == 2 else [ids[i]]
            full_row = user_id + wide_repr + ([ys[i]] if ys is not None else [])

            writer.writerow(full_row)

In [12]:
create_file('train', X_train_transformed, X_train, ids_train, y_train)
create_file('valid', X_valid_transformed, X_valid, ids_valid, y_valid)

In [10]:
def get_train_record_defaults():
    zf = tf.zeros(shape=(1,), dtype=tf.float32)

    # the +2 part is because of id and target
    return [zf] * (X_train_transformed.shape[1] + X_train.shape[1] + 2)


def parse_train_batch(tf_string: str):
    data = tf.io.decode_csv(tf_string, get_train_record_defaults())
    features = data[1:-1]
    labels = data[-1]
    features = tf.stack(features, axis=-1)

    return features, labels


def get_train_batched_dataset(batch_size: int, data_path: str) -> tf.data.Dataset:
    return tf.data.TextLineDataset([data_path])\
        .skip(1)\
        .batch(batch_size)\
        .map(parse_train_batch)

In [11]:
train_dataset = get_train_batched_dataset(64, os.path.join('data', 'train_enhanced.csv'))
valid_dataset = get_train_batched_dataset(64, os.path.join('data', 'valid_enhanced.csv'))

In [15]:
regressor_enhanced = keras.Sequential([
    SwapNoise(ratio=.1, col_to_apply=[X_train_transformed.shape[1] + idx for idx in range(X_train.shape[1])]),
    keras.layers.Dense(512),
    keras.layers.PReLU(),
    keras.layers.Dropout(.2),
    keras.layers.Dense(512),
    keras.layers.PReLU(),
    keras.layers.Dropout(.2),
    keras.layers.Dense(512),
    keras.layers.PReLU(),
    keras.layers.Dropout(.2),
    keras.layers.Dense(512),
    keras.layers.PReLU(),
    keras.layers.Dropout(.2),
    keras.layers.Dense(1),
    keras.layers.PReLU()
])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=30,
                                               min_delta=1e-6)
regressor_enhanced.compile(optimizer='adam', loss='mse')

history_enhanced = regressor_enhanced.fit(
    train_dataset, epochs=300, batch_size=64,
    callbacks=[
        early_stopping,
        keras.callbacks.ModelCheckpoint(
            filepath='saved_models/dnn_regressor_after_dae_enhanced{epoch}',
            save_best_only=True)],
    validation_data=valid_dataset)

Epoch 1/300
   3749/Unknown - 142s 37ms/step - loss: 0.8966INFO:tensorflow:Assets written to: saved_models\dnn_regressor_after_dae_enhanced1\assets
3750/3750 [==============================] - 171s 45ms/step - loss: 0.8966 - val_loss: 5.8594
Epoch 2/300
3750/3750 [==============================] - 174s 46ms/step - loss: 0.6555 - val_loss: 5.2708
Epoch 3/300
3750/3750 [==============================] - 174s 46ms/step - loss: 0.5992 - val_loss: 1.2945
Epoch 4/300
3750/3750 [==============================] - 177s 47ms/step - loss: 0.5535 - val_loss: 0.5753
Epoch 5/300
3750/3750 [==============================] - 176s 47ms/step - loss: 0.5354 - val_loss: 0.5200
Epoch 6/300
3750/3750 [==============================] - 174s 46ms/step - loss: 0.5293 - val_loss: 0.5291
Epoch 7/300
3750/3750 [==============================] - 182s 48ms/step - loss: 0.5234 - val_loss: 0.4934
Epoch 8/300
3750/3750 [==============================] - 176s 47ms/step - loss: 0.5186 - val_loss: 0.4963
Epoch 9/300
3750

In [12]:
enhanced_regressor = keras.models.load_model(os.path.join('saved_models', 'dnn_regressor_after_dae_enhanced10'))

In [14]:
X_test_transformed = rte.transform(test)
ridge_predictions = ridge.predict(X_test_transformed)
xgb_predictions = regressor.predict(test)

create_file('test', X_test_transformed, test, ids_test)

In [40]:
from tensorflow import Tensor


def parse_test_batch(tf_string: Tensor):
    zf = tf.zeros(shape=(1,), dtype=tf.float32)
    defaults = [zf] * (14375 + 14 + 2)
    data = tf.io.decode_csv(tf_string, defaults)
    features = data[1:]
    features = tf.stack(features, axis=-1)

    return features


def get_test_batched_dataset(batch_size: int, data_path: str) -> tf.data.Dataset:
    return tf.data.TextLineDataset([data_path])\
        .skip(1)\
        .batch(batch_size)\
        .map(parse_test_batch)

In [41]:
test_dataset = get_test_batched_dataset(64, os.path.join('data', 'test_enhanced.csv'))
nn_predictions = enhanced_regressor.predict(test_dataset)

3125/3125 [==============================] - 83s 25ms/step


In [45]:
def vote():
    reshaped_ridge_predictions = ridge_predictions.reshape((ridge_predictions.shape[0], 1))
    reshaped_xgb_predictions = xgb_predictions.reshape((xgb_predictions.shape[0], 1))
    avg = (reshaped_ridge_predictions + reshaped_xgb_predictions) / 2

    pd\
        .DataFrame(np.column_stack([pd.read_csv('data/test.csv')[['id']], avg]), columns=['id', 'target'])\
        .astype({'id': int})\
        .to_csv(os.path.join('submissions', 'ensemble2.csv'), index=False)


vote()